In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Extract cvs into DataFrame
staffing_file = "Resources/hospital-staffing-2009-2013-.csv"
staffing_file2 = "Resources/hospital-staffing-2009-2013-2.csv"
staffing_df = pd.read_csv(staffing_file)
staffing_df2 = pd.read_csv(staffing_file2)

In [3]:
staffing_df.head()

,Year,Facility Number,Facility Name,Begin Date,End Date,County Name,Type of Control,Hours Type,Productive Hours,Productive Hours per Adjusted Patient Day
0,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Management & Supervision,63558,1.17
1,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Technician & Specialist,163706,3.02
2,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Registered Nurse,180034,3.32
3,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Licensed Vocational Nurse,22323,0.41
4,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Aides & Orderlies,97205,1.79


In [4]:
staffing_df2.head()

,Year,Facility Number,Facility Name,Begin Date,End Date,County Name,Type of Control,Hours Type,Productive Hours,Productive Hours per Adjusted Patient Day
0,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Management & Supervision,63558,1.17
1,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Technician & Specialist,163706,3.02
2,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Registered Nurse,180034,3.32
3,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Licensed Vocational Nurse,22323,0.41
4,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Aides & Orderlies,97205,1.79


In [5]:
#1. COMPARE BOTH DATASETS TO IDENTIFY IF THEY ARE SAME OR DIFFERENT:
# resource: https://stackoverflow.com/questions/20225110/comparing-two-dataframes-and-getting-the-differences/52047609#52047609
#identify if the columns heading are equal
if staffing_df.columns.all() == staffing_df2.columns.all():
    print('columns in dfs are same')
else:
    print('columns in dfs differ')
#Compare dfs: concat dfs and drop all duplicates to get the df of differences
df_diff = pd.concat([staffing_df,staffing_df2]).drop_duplicates(keep=False)
print(f'number of different rows in dfs: {len(df_diff)}')
df_diff
#As the datasest are identical, we use further the staffing_df to work with. We can drop staffing_df2


columns in dfs are same
number of different rows in dfs: 0


,Year,Facility Number,Facility Name,Begin Date,End Date,County Name,Type of Control,Hours Type,Productive Hours,Productive Hours per Adjusted Patient Day


In [6]:
#2. CHECK FOR DUPLICATED ROWS/DATA IN DF: No identified
print(f"Number if duplicated rows in df: {len(staffing_df[staffing_df.duplicated(keep='first') == True])}")

staffing_df[staffing_df.duplicated(keep='first') == True]

Number if duplicated rows in df: 0


,Year,Facility Number,Facility Name,Begin Date,End Date,County Name,Type of Control,Hours Type,Productive Hours,Productive Hours per Adjusted Patient Day


In [7]:
#check the df for datatype and NaN values
staffing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37604 entries, 0 to 37603
Data columns (total 10 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Year                                       37604 non-null  int64  
 1   Facility Number                            37519 non-null  float64
 2   Facility Name                              37519 non-null  object 
 3   Begin Date                                 37519 non-null  object 
 4   End Date                                   37519 non-null  object 
 5   County Name                                37604 non-null  object 
 6   Type of Control                            37519 non-null  object 
 7   Hours Type                                 37604 non-null  object 
 8   Productive Hours                           37604 non-null  int64  
 9   Productive Hours per Adjusted Patient Day  37417 non-null  float64
dtypes: float64(2), int64(2

In [8]:
# 3. DROP ALL ROWS WHERE WITH NULL VALUE FOR 'FACILITY NUMBER':
# Based on the review of rows that includes NaN values we identified that all rows with NaN Facility Number are marked as county = statewide
# and therefore not required.

# - identify all rows with N/A data and review it
staffing_df[staffing_df.isnull().any(axis=1)]
# - drop all rows with Facility nUmber as NaN
staffing_df.dropna(subset=['Facility Number'], inplace=True)
staffing_df

,Year,Facility Number,Facility Name,Begin Date,End Date,County Name,Type of Control,Hours Type,Productive Hours,Productive Hours per Adjusted Patient Day
0,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Management & Supervision,63558,1.17
1,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Technician & Specialist,163706,3.02
2,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Registered Nurse,180034,3.32
3,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Licensed Vocational Nurse,22323,0.41
4,2009,106010735.0,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Aides & Orderlies,97205,1.79
...,...,...,...,...,...,...,...,...,...,...
37582,2013,106580996.0,RIDEOUT MEMORIAL HOSPITAL,07/01/2012,06/30/2013,Yuba,Non-Profit,Ancillary Cost Centers,769308,9.46
37583,2013,106580996.0,RIDEOUT MEMORIAL HOSPITAL,07/01/2012,06/30/2013,Yuba,Non-Profit,Education Cost Centers,0,0.00
37584,2013,106580996.0,RIDEOUT MEMORIAL HOSPITAL,07/01/2012,06/30/2013,Yuba,Non-Profit,General Services Cost Centers,250731,3.08
37585,2013,106580996.0,RIDEOUT MEMORIAL HOSPITAL,07/01/2012,06/30/2013,Yuba,Non-Profit,Fiscal Services Cost Centers,152364,1.87


In [9]:
staffing_df.count()

Year                                         37519
Facility Number                              37519
Facility Name                                37519
Begin Date                                   37519
End Date                                     37519
County Name                                  37519
Type of Control                              37519
Hours Type                                   37519
Productive Hours                             37519
Productive Hours per Adjusted Patient Day    37332
dtype: int64

In [10]:
#4 REPLACE NaN VALUES IN 'Productive Hours per Adjusted Patient Day' with zero:
staffing_df[staffing_df.isnull().any(axis=1)].count()

Year                                         187
Facility Number                              187
Facility Name                                187
Begin Date                                   187
End Date                                     187
County Name                                  187
Type of Control                              187
Hours Type                                   187
Productive Hours                             187
Productive Hours per Adjusted Patient Day      0
dtype: int64

In [11]:
staffing_df.fillna(0, inplace=True)
staffing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37519 entries, 0 to 37586
Data columns (total 10 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Year                                       37519 non-null  int64  
 1   Facility Number                            37519 non-null  float64
 2   Facility Name                              37519 non-null  object 
 3   Begin Date                                 37519 non-null  object 
 4   End Date                                   37519 non-null  object 
 5   County Name                                37519 non-null  object 
 6   Type of Control                            37519 non-null  object 
 7   Hours Type                                 37519 non-null  object 
 8   Productive Hours                           37519 non-null  int64  
 9   Productive Hours per Adjusted Patient Day  37519 non-null  float64
dtypes: float64(2), int64(2

In [12]:
# check the counts of unique values per columns - facility number and facility name have different number of unique values
# the further analysis is needed
staffing_df.nunique()

Year                                             5
Facility Number                                464
Facility Name                                  484
Begin Date                                      60
End Date                                        64
County Name                                     56
Type of Control                                  5
Hours Type                                      17
Productive Hours                             29085
Productive Hours per Adjusted Patient Day     1970
dtype: int64

In [13]:
num_name_check = staffing_df.copy()
num_name_check.drop_duplicates(subset=['Facility Number', 'Facility Name'], inplace=True)
num_name_issues = num_name_check[num_name_check['Facility Number'].duplicated(keep=False)].sort_values(by='Facility Number')
num_name_issues

,Year,Facility Number,Facility Name,Begin Date,End Date,County Name,Type of Control,Hours Type,Productive Hours,Productive Hours per Adjusted Patient Day
136,2009,106010887.0,KINDRED HOSPITAL - SAN FRANCISCO BAY AREA,01/01/2009,12/31/2009,Alameda,Investor,Management & Supervision,37185,1.90
22814,2012,106010887.0,KINDRED HOSPITAL SAN FRANCISCO BAY AREA,01/01/2012,12/31/2012,Alameda,Investor,Management & Supervision,51909,3.18
734,2009,106100797.0,SIERRA KINGS DISTRICT HOSPITAL,07/01/2008,06/30/2009,Fresno,District,Management & Supervision,66727,4.51
30957,2013,106100797.0,ADVENTIST MEDICAL CENTER - REEDLEY,01/01/2013,12/31/2013,Fresno,Non-Profit,Management & Supervision,23891,1.38
1453,2009,106190053.0,ST. MARY MEDICAL CENTER,07/01/2008,06/30/2009,Los Angeles,Non-Profit,Management & Supervision,155698,1.92
24140,2012,106190053.0,ST. MARY MEDICAL CENTER - LONG BEACH,07/01/2011,06/30/2012,Los Angeles,Non-Profit,Management & Supervision,133792,1.87
9089,2010,106190053.0,ST. MARY MEDICAL CENTER - LOS ANGELES,07/01/2009,06/30/2010,Los Angeles,Non-Profit,Management & Supervision,142415,1.80
2272,2009,106190517.0,TARZANA MEDICAL CENTER,01/01/2009,12/31/2009,Los Angeles,Investor,Management & Supervision,136159,1.75
9888,2010,106190517.0,PROVIDENCE TARZANA MEDICAL CENTER,01/01/2010,12/31/2010,Los Angeles,Non-Profit,Management & Supervision,128766,1.63
10228,2010,106190687.0,SANTA MONICA-UCLA MEDICAL CENTER & ORTHOPAEDIC...,07/01/2009,06/30/2010,Los Angeles,Non-Profit,Management & Supervision,251646,2.61


In [14]:
# unify the hospital names:
staffing_df['Facility Name'] = staffing_df['Facility Name'].replace({
    'KINDRED HOSPITAL SAN FRANCISCO BAY AREA': 'KINDRED HOSPITAL - SAN FRANCISCO BAY AREA',
    'SIERRA KINGS DISTRICT HOSPITAL': 'ADVENTIST MEDICAL CENTER - REEDLEY',
    'ST. MARY MEDICAL CENTER': 'ST. MARY MEDICAL CENTER - LONG BEACH',
    'ST. MARY MEDICAL CENTER - LOS ANGELES': 'ST. MARY MEDICAL CENTER - LONG BEACH',
    'TARZANA MEDICAL CENTER': 'PROVIDENCE TARZANA MEDICAL CENTER',
    'SANTA MONICA-UCLA MEDICAL CENTER& ORTHOPAEDIC HOSPITAL': 'SANTA MONICA-UCLA MEDICAL CENTER & ORTHOPAEDIC HOSPITAL',
    'STAR VIEW ADOLESCENT PSYCHIATRIC HEALTH FACILITY- PHF': 'STAR VIEW ADOLESCENT PSYCHIATRIC HEALTH FACILITY - PHF',
    'MERCY MEDICAL CENTER MERCED-COMMUNITY CAMPUS': 'MERCY MEDICAL CENTER MERCED - COMMUNITY CAMPUS',
    'KINDRED HOSPITAL - BREA': 'KINDRED HOSPITAL BREA',
    'FOUNTAIN VALLEY REGIONAL HOSPITAL& MEDICAL CENTER- EUCLID': 'FOUNTAIN VALLEY REGIONAL HOSPITAL& MEDICAL CENTER - EUCLID',
    'WESTERN MEDICAL CENTER-ANAHEIM': 'WESTERN MEDICAL CENTER - ANAHEIM',
    'KINDRED HOSPITAL WESTMINSTER': 'KINDRED HOSPITAL - WESTMINSTER',
    'WESTERN MEDICAL CENTER-SANTA ANA': 'WESTERN MEDICAL CENTER - SANTA ANA',
    'TELECARE PLACER COUNTY P.H.F': 'TELECARE PLACER COUNTY - PHF',
    'SOUTHWEST HEALTHCARE SYSTEM-MURRIETA': 'SOUTHWEST HEALTHCARE SYSTEM - MURRIETA',
    'KINDRED HOSPITAL ONTARIO': 'KINDRED HOSPITAL - ONTARIO',
    'ST. MARY MEDICAL CENTER': 'ST. MARY MEDICAL CENTER - APPLE VALLEY',
    'SCRIPPS MEMORIAL HOSPITAL-LA JOLLA': 'SCRIPPS MEMORIAL HOSPITAL - LA JOLLA',
    'UNIVERSITY OF CALIF - SAN DIEGO MEDICAL CENTER': 'UNIVERSITY OF CALIFORNIA SAN DIEGO MEDICAL CENTER',
    'KAISER PERMANENTE - PHF - SANTA CLARA': 'KAISER PERMANENTE P.H.F. - SANTA CLARA',
    'KAISER PERMANENTE P.H.F.- SANTA CLARA': 'KAISER PERMANENTE P.H.F. - SANTA CLARA',
    'KAISER PERMANENTE P.H.F.-SANTA CLARA': 'KAISER PERMANENTE P.H.F. - SANTA CLARA'
})

In [15]:
# check unique count
staffing_df.nunique()

Year                                             5
Facility Number                                464
Facility Name                                  463
Begin Date                                      60
End Date                                        64
County Name                                     56
Type of Control                                  5
Hours Type                                      17
Productive Hours                             29085
Productive Hours per Adjusted Patient Day     1970
dtype: int64

In [16]:
num_name_check2 = staffing_df.copy()
num_name_check2.drop_duplicates(subset=['Facility Number', 'Facility Name'], inplace=True)
num_name_issues2 = num_name_check2[num_name_check2['Facility Number'].duplicated(keep=False)].sort_values(by='Facility Number')
num_name_issues2

,Year,Facility Number,Facility Name,Begin Date,End Date,County Name,Type of Control,Hours Type,Productive Hours,Productive Hours per Adjusted Patient Day
1453,2009,106190053.0,ST. MARY MEDICAL CENTER - APPLE VALLEY,07/01/2008,06/30/2009,Los Angeles,Non-Profit,Management & Supervision,155698,1.92
9089,2010,106190053.0,ST. MARY MEDICAL CENTER - LONG BEACH,07/01/2009,06/30/2010,Los Angeles,Non-Profit,Management & Supervision,142415,1.80


In [19]:
num_name_check4 = staffing_df.copy()
num_name_check4.drop_duplicates(subset=['Facility Number', 'Facility Name'], inplace=True)
num_name_issues4 = num_name_check2[num_name_check4['Facility Number'].duplicated(keep=False)].sort_values(by='Facility Number')
num_name_issues4

C:\Users\zuzan\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,Facility Number,Facility Name,Begin Date,End Date,County Name,Type of Control,Hours Type,Productive Hours,Productive Hours per Adjusted Patient Day
1436,2009,106190052.0,BARLOW HOSPITAL,09/01/2008,08/31/2009,Los Angeles,Non-Profit,Management & Supervision,68210,3.10
1453,2009,106190053.0,ST. MARY MEDICAL CENTER - APPLE VALLEY,07/01/2008,06/30/2009,Los Angeles,Non-Profit,Management & Supervision,155698,1.92
9089,2010,106190053.0,ST. MARY MEDICAL CENTER - LONG BEACH,07/01/2009,06/30/2010,Los Angeles,Non-Profit,Management & Supervision,142415,1.80


In [56]:
#5. DATA TYPES CORRECTION:
# - facility number should be int64
staffing_df['Facility Number']=staffing_df['Facility Number'].astype('int64')
staffing_df.dtypes

Year                                           int64
Facility Number                                int64
Facility Name                                 object
Begin Date                                    object
End Date                                      object
County Name                                   object
Type of Control                               object
Hours Type                                    object
Productive Hours                               int64
Productive Hours per Adjusted Patient Day    float64
dtype: object

In [57]:
#6. RESET THE INDEX
staffing_df.reset_index(drop=True, inplace=True)
staffing_df

,Year,Facility Number,Facility Name,Begin Date,End Date,County Name,Type of Control,Hours Type,Productive Hours,Productive Hours per Adjusted Patient Day
0,2009,106010735,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Management & Supervision,63558,1.17
1,2009,106010735,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Technician & Specialist,163706,3.02
2,2009,106010735,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Registered Nurse,180034,3.32
3,2009,106010735,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Licensed Vocational Nurse,22323,0.41
4,2009,106010735,ALAMEDA HOSPITAL,07/01/2008,06/30/2009,Alameda,District,Aides & Orderlies,97205,1.79
...,...,...,...,...,...,...,...,...,...,...
37514,2013,106580996,RIDEOUT MEMORIAL HOSPITAL,07/01/2012,06/30/2013,Yuba,Non-Profit,Ancillary Cost Centers,769308,9.46
37515,2013,106580996,RIDEOUT MEMORIAL HOSPITAL,07/01/2012,06/30/2013,Yuba,Non-Profit,Education Cost Centers,0,0.00
37516,2013,106580996,RIDEOUT MEMORIAL HOSPITAL,07/01/2012,06/30/2013,Yuba,Non-Profit,General Services Cost Centers,250731,3.08
37517,2013,106580996,RIDEOUT MEMORIAL HOSPITAL,07/01/2012,06/30/2013,Yuba,Non-Profit,Fiscal Services Cost Centers,152364,1.87
